# 1. Load Data

### 1.1 Imports

In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/04 12:28:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### 1.2 Load Data

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, inferSchema=True)
home_sales.show(5)

+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|               date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08 00:00:00|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13 00:00:00|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12 00:00:00|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16 00:00:00|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08 00:00:00|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+-------

### 1.3 Create a temporary view of the DataFrame.

In [4]:
home_sales.createOrReplaceTempView("home_sales")

In [5]:
home_sales.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



# 2. Data Analysis

### 2.1 What is the average price for a four bedroom house sold in each year rounded to two decimal places?

In [6]:
%%time
spark.sql(" \
            select YEAR(date) as year, \
                   ROUND(AVG(price), 2) as avg_price \
            from home_sales \
            where bedrooms = 4 \
            group by 1 \
            order by 1 ").show()

+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+

CPU times: user 2.45 ms, sys: 1.49 ms, total: 3.94 ms
Wall time: 2.29 s


### 2.2 What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

In [7]:
%%time
spark.sql(" \
            select date_built as year_built, \
                   ROUND(AVG(price), 2) as avg_price \
            from home_sales \
            where bedrooms = 3 \
                and bathrooms = 3 \
            group by 1 \
            order by 1 ").show()

+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+

CPU times: user 2.41 ms, sys: 1.1 ms, total: 3.51 ms
Wall time: 1.5 s


###  2.3 What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors, and are greater than or equal to 2,000 square feet rounded to two decimal places?

In [8]:
%%time
spark.sql(" \
            select date_built as year_built, \
                   ROUND(AVG(price), 2) as avg_price \
            from home_sales \
            where bedrooms = 3 \
                and bathrooms = 3 \
                and floors = 2 \
                and sqft_living >= 2000 \
            group by 1 \
            order by 1 ").show()

+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+

CPU times: user 2.79 ms, sys: 1.58 ms, total: 4.37 ms
Wall time: 1.28 s


### 2.4 What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than or equal to $350,000? Although this is a small dataset, determine the run time for this query.

In [9]:
start_time = time.time()

spark.sql(" \
            select view, \
                   ROUND(AVG(price), 2) as avg_price \
            from home_sales \
            where price >= 350000 \
            group by 1 \
            order by 1 desc ").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 1.0917272567749023 seconds ---


# 3. Cache and Re-Run Data Analysis

### 3.1 Cache the the temporary table home_sales.

In [10]:
spark.sql("cache table home_sales")

DataFrame[]

In [11]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

### 3.2 Using the cached data, run the query that filters out the view ratings with average price greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

In [12]:
start_time = time.time()

spark.sql(" \
            select view, \
                   ROUND(AVG(price), 2) as avg_price \
            from home_sales \
            where price >= 350000 \
            group by 1 \
            order by 1 desc ").show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.9597649574279785 seconds ---


# 4. Partition Data and Re-Run Data Analysis

### 4.1 Partition by the "date_built" field on the formatted parquet home sales data 

In [13]:
home_sales.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

### 4.2 Read the formatted parquet data.

In [14]:
p_home_sales=spark.read.parquet('home_sales_partitioned')
p_home_sales.show(5)

+--------------------+-------------------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|               date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+-------------------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06 00:00:00|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09 00:00:00|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08 00:00:00|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04 00:00:00|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17 00:00:00|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
+-------

### 4.3 Create a temporary table for the parquet data.

In [15]:
p_home_sales.createOrReplaceTempView("p_home_sales")

In [16]:
p_home_sales.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)
 |-- date_built: integer (nullable = true)



### 4.4 Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 with the parquet DataFrame. Round your average to two decimal places. Determine the runtime and compare it to the cached version. 

In [17]:
start_time = time.time()

spark.sql(" \
            select view, \
                   ROUND(AVG(price), 2) as avg_price \
            from p_home_sales \
            where price >= 350000 \
            group by 1 \
            order by 1 desc ").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 2.032303810119629 seconds ---


# 5. Uncache Data

In [18]:
spark.sql("uncache table home_sales")

DataFrame[]

In [19]:
# Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False

23/06/04 12:36:26 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 275437 ms exceeds timeout 120000 ms
23/06/04 12:36:26 WARN SparkContext: Killing executors is not supported by current scheduler.
